<a href="https://colab.research.google.com/github/SGRGit/gwOsc/blob/FinalCode/DataPrepForBlipGlitchClassification_N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [0]:
import gwpy
import numpy as np
import matplotlib.pyplot as plt
import math

****Analysis with LIGO data for Glitch Recognition using Q-Transforms****

In [0]:
#Import Blip Glitch o1 Data from gspy in Pandas Dataframe
import pandas as pd
raw_data = pd.read_csv("/content/gdrive/My Drive/gspy_blip_o1.csv")

glitch_data = raw_data[['GPStime','ifo', 'peakFreq', 'snr', 'amplitude', 'centralFreq','duration']]

In [0]:
import os
base_dir = "gdrive/My Drive/Colab Notebooks/GlitchNoiseClassification/"
seta = 'Glitch'
setb = 'Noise'

data_dir = os.path.join(base_dir, 'Data')
seta_data_dir = os.path.join(data_dir, seta)
seta_data_fnnames = os.listdir(seta_data_dir)

setb_data_dir = os.path.join(data_dir, setb)
setb_data_fnnames = os.listdir(setb_data_dir)

In [0]:
noise_data_cln =  glitch_data.iloc[2750 : ]

In [0]:
noise_data_cln.reset_index(inplace = True)

In [20]:
noise_data_cln

,index,GPStime,ifo,peakFreq,snr,amplitude,centralFreq,duration
0,2750,1.129458e+09,L1,512.608,58.620,1.990000e-19,1521.968,0.250
1,2751,1.129511e+09,H1,276.750,26.288,4.600000e-22,1212.732,0.250
2,2752,1.129564e+09,H1,121.663,22.528,2.310000e-22,204.324,0.281
3,2753,1.135883e+09,H1,262.065,9.676,1.550000e-22,488.121,0.016
4,2754,1.132347e+09,H1,170.654,9.030,6.190000e-23,3268.060,0.105
...,...,...,...,...,...,...,...,...
365,3115,1.127063e+09,H1,180.507,8.031,1.070000e-22,222.291,0.125
366,3116,1.128756e+09,H1,312.908,8.195,1.010000e-22,312.274,0.344
367,3117,1.128127e+09,H1,53.485,8.049,1.140000e-21,83.413,0.094
368,3118,1.134499e+09,L1,211.477,9.123,8.990000e-23,578.150,0.023


In [71]:
import os
files = os.listdir(seta_data_dir)
len(files)

2501

In [21]:
import time
from gwpy.timeseries import TimeSeries
epoch_noise = np.empty(370)
ifo_noise = []
images_noise = []

for i in range(0, 370, 1):
  epoch_noise[i] = int(noise_data_cln['GPStime'][i])
  ifo_noise.append(noise_data_cln['ifo'][i])
#print(ifo_noise, epoch_noise)

for a in range(0, 370, 1):
  print(a)
  I = TimeSeries.fetch_open_data(ifo_noise[a], epoch_noise[a]-4, epoch_noise[a]-1).q_transform()
  images_noise.append(I)
  fig, ax = plt.subplots(1, 1, figsize = (10, 5))
  ax.imshow(I)
  ax.set_xscale('seconds')
  ax.set_yscale('log')
  ax.set_ylabel('Frequency [Hz]')
  ax.grid(True, axis='y', which='both')
  ax.colorbar(cmap='viridis', label='Normalized energy')
  ax.set_title('Q-Transform for Noise at %s on GPS %d' %(ifo_noise[a], epoch_noise[a]))
  plt.savefig(os.path.join(setb_data_dir,'Q-Transform for Noise at %s on GPS %d' %(ifo_noise[a], epoch_noise[a])))
  plt.close()
  time.sleep(0.1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


UnboundLocalError: ignored